In [41]:
import copy as cp
import numpy as np

In [42]:
class Puzzle:
    RED = 1
    GREEN = 2
    YELLOW = 3
    SIZE = 6
    def __init__(self):
        self.board = np.random.choice([Puzzle.RED, Puzzle.GREEN,Puzzle.YELLOW],(Puzzle.SIZE,Puzzle.SIZE))
    def __str__(self):
        return str(self.board)
    
    def goal_test(self):
        if np.all(self.board == 1) or np.all(self.board == 2) or np.all(self.board == 3):
            return True
        return False
    
    # Actions
    def fill_green(self):
        visited = np.zeros_like(self.board)
        child = cp.deepcopy(self)
        child.board[0][0] = Puzzle.GREEN
        child = self.fill_color(node=child, color=Puzzle.GREEN, visited=visited, row=0, col=0)
        return child
    
    def fill_color(self, node, color, visited, row, col):
        # check left, right, up, down if they have green color
        # if yes then move to that cell and recursively check it's neighbours
        if row < 0 or row > Puzzle.SIZE or col < 0 or col > Puzzle.SIZE:
            return node
        elif visited[row][col] == 1: # already visited
            return node
        elif visited[row][col] != Puzzle.GREEN:
            return node
        
        visited[row][col] = 1
        node.board[row][col] = color
        node = self.fill_color(node=node, color=Puzzle.GREEN, visited=visited, row=row+1, col=col)
        node = self.fill_color(node=node, color=Puzzle.GREEN, visited=visited, row=row-1, col=col)
        node = self.fill_color(node=node, color=Puzzle.GREEN, visited=visited, row=row, col=col+1)
        node = self.fill_color(node=node, color=Puzzle.GREEN, visited=visited, row=row, col=col-1)
        return node
        



In [43]:
board = Puzzle()
print(board)
board.goal_test()

[[2 2 3 2 1 2]
 [1 3 3 3 1 3]
 [1 3 1 1 3 1]
 [1 3 1 1 1 1]
 [1 1 3 3 2 1]
 [3 1 2 2 1 3]]


False

In [44]:
child = board.fill_green()
print(child)

[[2 2 3 2 1 2]
 [1 3 3 3 1 3]
 [1 3 1 1 3 1]
 [1 3 1 1 1 1]
 [1 1 3 3 2 1]
 [3 1 2 2 1 3]]
